# Hierarchical Non-linear Panel Models
---
## Motivation
- This notebook is a **cookbook** of hierarchical models built using `pymc3`.
- `pymc3` is a Python package for Bayesian statistical modeling.
- `pymc3`'s API design falls under the "probablistic programming" paradigm where:
    - Probalistic models are specified
    - Inference is performed for these models automatically (using MCMC methods or variational inference)
- If you recall during Rachael's "Fixed Effects and Random Effects" lecture, a mixed effects model balances the bias/variance tradeoff between random effects and fixed effects by combining both effects together.
- I use the term "mixed effect" to differentiate it from the "Bayesian hierarchical model".
- The hierarchical model is similar to the mixed effect model except for one key difference: priors distribution are given to the random effects' hyperparameters.
- I decided to make this notebook because conditional logit in `statsmodels` is a work-in-progress and throws out an error (https://github.com/statsmodels/statsmodels/issues/5904) that has yet to be resolved.
- The power of probablistic programming, however, allows us to define complex statistical models from first-principles and perform Bayesian inference on them.

## Overview (and pre-analysis plan)
- Build Bayesian models using `pymc3`
- Perform Bayesian inference on these models using MCMC sampling
- Models include:
    1. Pooled logit
    2. Hierarchical logit
    3. Hierarchical logit
    4. LPM
    5. Hierachical LPM
    6. Hierachical LPM
- Perform posterior predictive checks using `arviz`
- Model comparison using WAIC (widely applicable information criterion)

## Fixed Effects vs Random Effects
---


##  Mixed Effects vs Bayesian Hierachical Models
---

In [1]:
import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf

from statsmodels.discrete.conditional_models import ConditionalLogit

# 1. Data prep

In [2]:
# Load data (from http://www.princeton.edu/~otorres/LogitR101.pdf )

data = pd.read_stata('http://dss.princeton.edu/training/Panel101.dta')
data

,country,year,y,y_bin,x1,x2,x3,opinion,op
0,A,1990,1.342788e+09,1.0,0.277904,-1.107956,0.282554,Str agree,1.0
1,A,1991,-1.899661e+09,0.0,0.320685,-0.948720,0.492538,Disag,0.0
2,A,1992,-1.123436e+07,0.0,0.363466,-0.789484,0.702523,Disag,0.0
3,A,1993,2.645775e+09,1.0,0.246144,-0.885533,-0.094391,Disag,0.0
4,A,1994,3.008335e+09,1.0,0.424623,-0.729768,0.946131,Disag,0.0
...,...,...,...,...,...,...,...,...,...
65,G,1995,1.323696e+09,1.0,1.087186,-1.409817,2.829808,Str disag,0.0
66,G,1996,2.545242e+08,1.0,0.781075,-1.328000,4.278224,Str agree,1.0
67,G,1997,3.297033e+09,1.0,1.257879,-1.577367,4.587326,Disag,0.0
68,G,1998,3.011821e+09,1.0,1.242777,-1.601218,6.113762,Disag,0.0


In [24]:
# Seperate exogenous columns, endogenous columns, 
# country column, and year column

exog = data[['x1', 'x2', 'x3']]
endog = data['y_bin'].astype(int)
countries = data['country']
years = data['year']